### Script de modelagem direta da bacia de barreirinhas - dados grav adquiridos por Nelson Delimar

In [ ]:
# importando pacotes necessarios:
import numpy as np # pacote numerico
import pylab as py 
#from numpy.linalg import inv # pacote de algebra linear
import matplotlib.pyplot as plt # pacote de plotagem
#import pandas as pd # pacode de tratamento de dados 
#from mpl_toolkits.basemap import Basemap
#from matplotlib.patches import Polygon
#from scipy.interpolate import Rbf # radial basis functions technique
#para plotar as figuras nas celulas do jupyter notebook:

#%matplotlib inline
%matplotlib tk
#plt.plot ([1,2])

### Importando os pacotes com as nossas funcoes:

In [ ]:
from modules.iplots import pick_points # para fazer o plot interativo
from modules.gravity import grav2D_anom # funcao que calcula dados de gravidade (rao 1994)

### Leitura dos dados reais do perfil de Barreirinhas:

In [ ]:
# leitura no script do pc do bijani:
#x_utm, y_utm, elev, gz = np.loadtxt('../../dados_reais/Barreirinhas/barreirinhas_perfil.txt', skiprows=1 , usecols = (0,1,2,3), unpack = True)
x_utm, y_utm, elev, gz = np.loadtxt('../dados_reais/Barreirinhas/barreirinhas_perfil.txt', 
                                    skiprows=1 , usecols = (0,1,2,3), unpack = True)

# leitura no script no pc do mathews:
#x_utm, y_utm, elev, gz = np.loadtxt('../dados_reais/barreirinhas_perfil.txt', skiprows=1 , usecols = (0,1,2,3), unpack = True)

#alterando o sinal de elev devido a mudança da referencia do eixo z (positivo para baixo):
elev = -elev                                   

In [ ]:
# Plot dos dados para verificação:
%matplotlib tk
plt.figure(figsize=(12,9))
plt.plot(x_utm, gz,'*r')
plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Gravity Anomaly (mGal)', fontsize = 14)

### Definicao da area de plot do modelo:

In [ ]:
area= []
xmin = np.min (x_utm)
xmax = np.max (x_utm)
zmin = np.min ( (elev) )
zmax = 4.0 #profundidade da bacia em km
print zmin
area = [xmin, xmax, zmin, zmax]

#print area

### Criacao do modelo de bacia atraves dos clicks na area de plot: 

In [ ]:
axes = plt.figure().add_subplot(1,1,1)
xv,zv = pick_points(area, axes, marker='o', color='k', size=8, xy2ne=False)

### calculo da anomalia de gravidade produzido pelo modelo de bacia (artigo do Rao et. al., 1994):

In [ ]:
# info da propriedade fisica:
delta_rho = -1.90 #(g/cm³)#Densidade do arenito
beta = 3.12
# convertendo para SI(kg/m³):
rho_ref = 3.0 # densidade do embasamento (checar!)
rho = delta_rho + rho_ref
rho = rho * 1000.0 
print rho

In [ ]:
#Criacao de variaveis necessarias para o calculo da anomalia:
nv = np.size (xv)
#print nv
nper = np.size (x_utm)
#print nper
# calculo da anomalia grav atraves de rao1994:
xd = np.zeros( (nv,) )
zd = np.zeros( (nv,) )
grav = np.zeros( (nper,) )
for i in range(nper):
    for j in range(nv):
        xd[j] = ( xv[j] - x_utm[i])
        zd[j] = (zv[j] - elev[i])
    # chamando a funcao do rao 1994:
    grav[i] = grav2D_anom(xd,zd,delta_rho,beta)
    
print grav

### Plot do modelo atual + os dados reais para comparação:

In [ ]:
plt.clear()
plt.plot(x_utm,gz,'.k', label='observed data')
plt.plot(x_utm,grav,'.r', label='predicted data')
plt.legend()
plt.grid()
plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Gravity Anomaly (mGal)', fontsize = 14)
plt.show()

# Criacao de um branch para nao perder clicks promissores!!!!!!

In [ ]:
## criacao do branch --- variaveis guardadas para nao perder cliques promissores!
### SOMENTE RODAR ESSA CELULA EM CASO DE MELHORIAS SIGNFICATIVAS NOS CLICKS!!!!
x_branch = np.copy(xv)
y_branch = np.copy(zv)
gz_branch = np.copy(grav)

### TODO:: Gerar um arquivo com as variaveis branch para backup de clicks promissoes!

In [ ]:
# ---- comparacao entre o dado branch vs o ultimo click-----:
plt.clear()
plt.plot(x_utm, grav,'.k', label='observed data')
plt.plot(x_utm, gz_branch,'.m', label='branch data')
plt.legend()
plt.grid()
plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Gravity Anomaly (mGal)', fontsize = 14)
plt.show()